Installing required packages

In [1]:
!pip install simpletransformers
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importing libraries

In [2]:
import gradio as gr
import re

Setting Language generation model (GPT2) arguments Max words:40

In [3]:
from simpletransformers.language_generation import LanguageGenerationModel,LanguageGenerationArgs
Language_gen_args = LanguageGenerationArgs()
Language_gen_args.max_length=40
Language_gen_args.use_cuda=False

Getting our  best finetuned GPT2 Language generation model

In [4]:
model=LanguageGenerationModel("gpt2",'/content/drive/MyDrive/NLP_CommentaryDatasets/output/model',args=Language_gen_args,use_cuda=False)

Generate commentary for a sample input

In [23]:
output=model.generate("SIX runs")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sample generated commentary

In [24]:
output[0]

'SIX runs, slapped over cow corner! Slower ball, actually short of a length outside off. Enough room for him to free his arms and use the pace to muscle it over the fielder.1 run,'

Sometimes commentary can be generated for 2 events if initial generation is too short so retaining only the relevent commentary from the 1st event

In [26]:
result=output[0]
commentary=re.split(r"\.\s*",result)
for i in range (1,len(commentary)):
  if(re.match(r'[1-6]|FOUR|SIX|OUT|no\s\run',commentary[i])!="NONE"):
    index=i

result=commentary[0]
for i in range (1,index):
  result=result+". "+commentary[i]

result

'SIX runs, slapped over cow corner! Slower ball, actually short of a length outside off. Enough room for him to free his arms and use the pace to muscle it over the fielder'

Function to generate commentary similar to cricbuzz given user input of bowler, batsman, runs_scored and extras

In [8]:
def generatecommentary(bowler,batsman,runs_scored,extra):
  index = 1;
  prompt = str(bowler)+" to "+str(batsman)
  event = generateevent(runs_scored,extra)
  output=model.generate(event)

  result=output[0]
  commentary=re.split(r"\.\s*",result)
  for i in range (1,len(commentary)):
    if(re.match(r'[1-6]|FOUR|SIX|OUT|no|one\s\run',commentary[i])!="NONE"):
      index=i

  result=commentary[0]
  for i in range (1,index-1):
    result=result+". "+commentary[i]

  res=prompt+" "+result
  return res

Generating an event in format similar to cricbuzz given the number of runs scored and the extras of a ball

In [9]:
def generateevent(runs_scored,extra):
  if(runs_scored==0):
    event="no run"

  if((extra=="no extra" or extra=="no ball") and runs_scored!=0 ):
    if(extra=="no ball"):
      event="(no ball) "
    else:
      event = ""

    if(runs_scored=="1"):
      event +="one run"
    elif(runs_scored=="2" or runs_scored=="3" or runs_scored=="5"):
      event+=runs_scored+" runs"
    elif(runs_scored=="4"):
      event+="FOUR runs"
    elif(runs_scored=="6"):
      event+="SIX runs"
      
  else:
    if(runs_scored!="0" or runs_scored!="6"):
      event=runs_scored+" "+extra
  return event

Gradio inputs for generating commentary

In [10]:
input = [
    gr.inputs.Textbox(label="Bowler"),
    gr.inputs.Textbox(label="Batsman"),
    gr.Dropdown(["0", "1","2","3","4","5","6"], label="Runs scored (including extras)"),
    gr.Dropdown(["no extra","wide","no ball","leg bye"], label="Extra_type")
]

/usr/local/lib/python3.7/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Gradio UI for getting input, processing and showing result

In [28]:
commentarygenerator = gr.Interface(
    fn=generatecommentary,
    inputs=input,
    outputs=gr.outputs.Textbox(label="Generated Commentary")
)

/usr/local/lib/python3.7/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  "Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Launching the commentary generator

In [29]:
commentarygenerator.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().

Using Embedded Colab Mode (NEW). If you have issues, please use share=True and file an issue at https://github.com/gradio-app/gradio/
Note: opening the browser inspector may crash Embedded Colab Mode.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7fa8c5c3ef10>, 'http://127.0.0.1:7860/', None)